<a href="https://colab.research.google.com/github/simulate111/Computer-Vision-and-Sensor-Fusion/blob/main/Assignment_1_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import random
import os
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

In [2]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json file

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 600)

In [ ]:
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification

In [ ]:
!unzip muffin-vs-chihuahua-image-classification.zip -d /content/

In [ ]:
os.listdir('/content')

### Path system

In [ ]:
# you need the current working directory NB: works both windows and linux
current_working_directory = os.getcwd()
current_working_directory = os.path.dirname(current_working_directory)

# get the directory where I want to download the dataset
path_of_download = os.path.join(*['..', current_working_directory, 'Datasets', 'cookies_vs_chihuahua'])
print(f"[DIR] The directory of the current dataset is {path_of_download}")

### Dataset function

In [ ]:
# here let s do some functions that we can re-use also for other assignment
def load_the_data_and_the_labels(data_set_path: str, target_size: tuple or None = None):
    try:
        dataset, labels, name_of_the_labels = list(), list(), list()
        # let s loop here and we try to discover how many class we have
        for class_number, class_name in enumerate(os.listdir(data_set_path)):
            full_path_the_data = os.path.join(data_set_path, class_name)
            print(f"[WALK] I am walking into {full_path_the_data}")

            # add the list to nam _list
            name_of_the_labels.append(class_name)

            for single_image in os.listdir(f"{full_path_the_data}"):
                full_path_to_image = os.path.join(*[full_path_the_data, single_image])

                # add the class number
                labels.append(class_number)

                if target_size is None:
                    # let s load the image
                    image = tf.keras.utils.load_img(full_path_to_image)
                else:
                    image = tf.keras.utils.load_img(full_path_to_image, target_size=target_size)

                # transform PIL object in image
                image = tf.keras.utils.img_to_array(image)

                # add the image to the ds list
                dataset.append(image)

        return np.array(dataset, dtype='uint8'), np.array(labels, dtype='int'), name_of_the_labels
    except Exception as ex:
        print(f"[EXCEPTION] load the data and the labels throws exceptions {ex}")

### load train set

In [ ]:
#train_data_path = os.path.join(path_of_download, "train")
#train_dataset, train_labels, train_class_names = load_the_data_and_the_labels(train_data_path, (224, 224, 3))

In [ ]:
train_data_path = '/content/train'
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_data_path, image_size=(224, 224), batch_size=32, label_mode='binary')
print(f"Train dataset: {train_dataset}")
train_images = []
train_labels = []
train_class_names = train_dataset.class_names
for images, labels in train_dataset:
    train_images.append(images.numpy())
    train_labels.append(labels.numpy())

train_images = np.concatenate(train_images, axis=0)
train_labels = np.concatenate(train_labels, axis=0)

### load test set

In [ ]:
#test_data_path = os.path.join(path_of_download, "test")
#test_data_path ='/content/muffin-vs-chihuahua-image-classification/test'
#test_dataset, test_labels, test_class_names = load_the_data_and_the_labels(test_data_path, (224, 224, 3))

In [ ]:
test_data_path = '/content/test'
# Load the test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(test_data_path, image_size=(224, 224), batch_size=32, label_mode='binary')
print(f"Test dataset: {test_dataset}")
test_images = []
test_labels = []
test_class_names = test_dataset.class_names
for images, labels in test_dataset:
    test_images.append(images.numpy())
    test_labels.append(labels.numpy())

test_images = np.concatenate(test_images, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

### normalize the data

In [ ]:
#train_dataset = train_dataset / 255.0
#test_dataset = test_dataset / 255.0
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.3)

In [ ]:
print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")

### create a cnn with the following characteristics:
a.	Input layer
b.	Data augmentation, with random flip and random rotation.
c.	Two hidden layers each composed with the following characteristics: 16 conv2d units, max pooling 2d and batch normalization, the second one should have 24 conv2d units max pooling 2d and batch normalization.
d.	After this, add a flatten layer and a dense layer with 8 units
e.	Add the final classifier (a  dense layer) with the correct number of output and activation


In [ ]:
#Model
cnn_model = models.Sequential()

#a
cnn_model.add(layers.InputLayer(shape=(224, 224, 3)))

#b
cnn_model.add(layers.RandomFlip("horizontal"))
cnn_model.add(layers.RandomRotation(0.2))

#c
cnn_model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.BatchNormalization())

cnn_model.add(layers.Conv2D(24, (3, 3), activation='relu', padding='same'))
cnn_model.add(layers.MaxPooling2D((2, 2)))
cnn_model.add(layers.BatchNormalization())

#d
cnn_model.add(layers.Flatten())
cnn_model.add(layers.Dense(8, activation='relu'))

#e
cnn_model.add(layers.Dense(1, activation='sigmoid'))

### compile the model using Adam

In [ ]:
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Train the model with batch size 64 and epochs of 30

In [ ]:
cnn_model.fit(X_train, y_train, batch_size=64, epochs=30, validation_data=(X_test, y_test))

### Evaluate the model and report the accuracy.

In [ ]:
#print('Accuracy', cnn_model.evaluate(test_dataset, test_labels)[1])
# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Print the accuracy
print(f"Test Accuracy: {test_accuracy[1]:.4f}")


#### Make prediction with the test set and use a threshold of 0.5 as boundaries decision between the classes.

In [ ]:
predicted_classes = (cnn_model.predict(test_dataset) > 0.5).astype(int)

### plot confusion matrix and ROC curve

In [ ]:
ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(test_labels, predicted_classes), display_labels=test_class_names).plot()

In [ ]:
fpr, tpr, _ = roc_curve(test_labels, predicted_classes)
plt.plot(fpr, tpr)
print('ROC:', auc(fpr, tpr))

### Calcualte best tresthold

In [ ]:
precision, recall, thresholds = precision_recall_curve(test_labels, cnn_model.predict(test_dataset))
f1_scores = 2 * (precision * recall) / (precision + recall)
best_threshold_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_threshold_idx]

### Plot confusion matrix

In [ ]:
best_predicted_classes = (cnn_model.predict(test_dataset) > best_threshold).astype(int)
ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(test_labels, best_predicted_classes)).plot()